In [1]:
# Dependencies
from bs4 import BeautifulSoup
from lxml import html
import requests
import pymongo
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/justinmerryman/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [10]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
# Retrieve page with the requests module
#response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [13]:
news_title = soup.find_all('div', class_= "content_title")[1].text
print(news_title)
news_p = soup.find('div', class_= "article_teaser_body").text
print(news_p)

NASA's Perseverance Pays Off Back Home
Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.


In [12]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup2 = BeautifulSoup(response.text, 'html.parser')
#print(soup2.prettify())

In [7]:
news_title = soup2.find_all('div', class_= "content_title")[1].text
print(news_title)
news_p = soup2.find('div', class_= "article_teaser_body").text
print(news_p)



NASA InSight's ‘Mole' Ends Its Journey on Mars




AttributeError: 'NoneType' object has no attribute 'text'

In [14]:
url2 = 'https://space-facts.com/mars/'

In [15]:
tables = pd.read_html(url2)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [16]:
df = tables[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [17]:
df.to_html('table.html')

In [18]:
url3 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url3)

In [19]:
hemisphere_image_urls= [{}]

In [37]:
html = browser.html
soup3 = BeautifulSoup(html, 'html.parser')
links = soup3.find_all('a', class_='itemLink')
title = soup3.find_all('h2', class_='title')

browser.links.find_by_partial_text('/search/map/Mars/Viking/cerberus_enhanced').click()
print(links)

ElementDoesNotExist: no elements could be found with link by partial text "/search/map/Mars/Viking/cerberus_enhanced"

[]


In [34]:
html = browser.html
soup3 = BeautifulSoup(html, 'html.parser')
image_url = soup3.find_all('a', class_='itemLink')
title = soup3.find_all('h3')

print(image_url)
print(title)

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>, <a 